# Desafio: Preveja os usuários com alta chance de deixar seu Streaming

# Contexto - Introdução

Você trabalha em uma plataforma de streaming e a diretoria está preocupada com o alto índice de usuários cancelando as suas assinaturas. Eles acreditam que é possível prever se um usuário tem mais chance de deixar a plataforma antes que isso aconteça, e com base nessa informação tomar ações para reduzir o churn.

Seu objetivo é criar um modelo de classificação capaz de prever se um usuário tem mais chance de cancelar a sua assinatura na plataforma ou não. Para isso, a empresa forneceu uma base de dados em csv contendo dados sobre as contas dos clientes.

# Sobre os dados

Os dados fornecidos possuem informações sobre as contas dos clientes na plataforma de streaming, divididos entre contas Basic, Standard e Premium, onde cada uma oferece uma gama maior de serviços que a anterior.

| Coluna    | Descrição              | Tipo |
|-----------|------------------------|------|
| client_id   | Código de identificação do cliente | Int |
| age         | Idade do cliente | Int |
| gender      | Gênero do cliente | String |
| region      | Região de origem do cliente | String |
| subscription_days | Dias de assinatura ativa do cliente | Int |
| subscription_type | Tipo de conta | String |
| num_contents | Quantidade de conteúdos assistidos | Int |
| avg_rating | Avaliação média dos conteúdos da plataforma | Int |
| num_active_profiles | Número de perfis ativos na plataforma | Int |
| num_streaming_services | Quantidade de serviços de streaming que o cliente possui | Int |
| devices_connected | Quantidade de dispositivos conectados à conta | Int |
| churned | Se o cliente cancelou a conta ou não | Int |

# Importando as Bibliotecas

In [8]:
#Manipulação de Dados
import pandas as pd
import numpy as np

#Visualização
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots            import make_subplots

#Modelo
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

#Display
import warnings
from IPython.display            import Markdown

# Funções Auxiliares e Configurações

In [9]:
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.2f}'.format

def df_informations(df):
    # Cria um Dataframe com as informações relativas ao Dataset
    df_info = pd.DataFrame({'Not Null': df.notnull().sum(),
                            'Null': df.isnull().sum(),
                            'Perce Null': df.isnull().sum() / len(df),
                            'Unique': df.nunique(),
                            'Dtype': df.dtypes
                })
    
    # Cria outro Dataframe com todos os types do df
    df_dtype = df_info['Dtype'].value_counts().reset_index()
    df_dtype.columns = ['Dtype', 'Count']
    df_dtype['Perce'] = round(df_dtype['Count'] / df_dtype['Count'].sum(), 2)
    
    # Verificando se tem linhas duplicadas
    duplicado = df.duplicated().sum()
    mensagem = 'Não possui linhas duplicadas.' if duplicado == 0 else f"Tem {duplicado} linhas duplicadas, o que representa {round(duplicado/len(df)*100, 2)}% do total de linhas."

    # Texto automático das dimensões do df
    text = f'Dataset tem {df.shape[0]} linhas e {df.shape[1]} colunas. {mensagem} Sobre o dataset, temos:'

    # Colore os Dataframes
    df_info = df_info.style.background_gradient(cmap='jet', subset=['Perce Null']).format({'Perce Null': '{:.2%}'})
    df_dtype = df_dtype.style.background_gradient(cmap='YlGn', subset=['Perce']).format({'Perce': '{:.2%}'})

    # Texto final com Markdown
    display(Markdown("<H3 style='text-align:left;float:lfet;'>Informações sobre o Dataset"))
    display(Markdown(f'<H5> {text}'))
    display(df_info)
    display(Markdown("<H3 style='text-align:left;float:lfet;'>Sobre Dtypes, temos:"))
    display(df_dtype)
    display(Markdown("<H3 style='text-align:left;float:lfet;'>Estatística Descritiva"))
    display(df.describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]).T)

def plot_distribution_and_boxplot(df):
    # Obtém a lista de colunas numéricas
    num_cols = df.select_dtypes(include=[np.number]).columns
    
    # Cria a lista de títulos para os subplots
    subplot_titles = [title for col in num_cols for title in [f"Histograma - {col}", f"Boxplot - {col}"]]
    
    # Cria a figura com subplots
    fig = sp.make_subplots(rows=len(num_cols), cols=2, subplot_titles=subplot_titles)
    
    # Define cores
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown', 'pink', 'yellow', 'cyan', 'magenta', 
              'lime', 'lavender', 'maroon', 'navy', 'olive', 'teal', 'aqua', 'fuchsia', 'gray', 'silver']
    
    # Adiciona os gráficos à figura para cada coluna numérica
    for i, col in enumerate(num_cols, start=1):
        # Define a cor usando a lógica de ciclagem
        color = colors[i % len(colors)]
        
        # Adiciona o histograma
        fig.add_trace(go.Histogram(x=df[col], nbinsx=40, marker_color=color, 
                                    showlegend=False),
                      row=i, col=1)
        
        # Adiciona o boxplot
        fig.add_trace(go.Box(y=df[col], marker_color=color, showlegend=False),
                      row=i, col=2)
    
    # Configura o layout
    fig.update_layout(height=300*len(num_cols), title_text="Distribuição e Boxplot para cada variável numérica")
    fig.show()

def calculate_iqr(df):
    # Define um DataFrame vazio para armazenar os resultados
    iqr_df = pd.DataFrame(columns=['Coluna', 'Q1', 'Q3', 'IQR', 'Lower Bound', 'Upper Bound', 'Min', 'Max'])
    
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.number):  # Se a coluna for numérica
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            min_val = df[col].min() # valor mínimo
            max_val = df[col].max() # valor máximo
            # Cria um dicionário temporário com os dados
            temp_dict = {'Coluna': col, 'Q1': Q1, 'Q3': Q3, 'IQR': IQR, 
                         'Lower Bound': lower_bound, 'Upper Bound': upper_bound, 
                         'Min': min_val, 'Max': max_val}
            # Cria um DataFrame temporário a partir do dicionário
            temp_df = pd.DataFrame([temp_dict])
            # Concatena este DataFrame temporário com o DataFrame principal
            iqr_df = pd.concat([iqr_df, temp_df], ignore_index=True)
    return iqr_df

def plot_boxplots(df):
    # Calcula o número de linhas para os subplots
    num_rows = math.ceil(len(df.columns) / 2)

    # Cria a figura com subplots
    fig = make_subplots(rows=num_rows, cols=2)

    # Adiciona os boxplots à figura para cada coluna numérica
    for i, col in enumerate(df.columns, start=1):
        fig.add_trace(
            go.Box(
                y=df[col], 
                name=col, 
                boxmean=True, 
                showlegend=False
            ), 
            row=(i-1)//2 + 1, 
            col=((i-1)%2) + 1
        )

    # Atualiza o layout da figura
    fig.update_layout(height=400*num_rows, title_text="Boxplot de cada coluna numérica")

    # Mostra a figura
    fig.show()

def update_graph2(m1, m2, b):
    # Gerando a superfície de previsão
    x1_range = np.linspace(x1.min(), x1.max(), 100)
    x2_range = np.linspace(x2.min(), x2.max(), 100)
    x1_values, x2_values = np.meshgrid(x1_range, x2_range)
    y_pred = m1*x1_values + m2*x2_values + b

    # Criando o gráfico
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=x1.squeeze(), y=x2.squeeze(), z=y2.squeeze(), mode='markers', name='Dados'))
    fig.add_trace(go.Surface(x=x1_range, y=x2_range, z=y_pred, opacity=0.7, name='Superfície de Regressão'))

    fig.update_layout(  title="Demonstração de Regressão Linear Múltipla",
                        scene=dict(
                        xaxis_title='X1',
                        yaxis_title='X2',
                        zaxis_title='Y'
    ),
    #width=1000, # Altera a largura do gráfico
    height=750 # Altera a altura do gráfico
    )

    return fig


# Importando o Dataset

In [10]:
df = pd.read_csv('streaming_data.csv')
df

,Age,Gender,Time_on_platform,Devices_connected,Subscription_type,Num_streaming_services,Num_active_profiles,Avg_rating,Churned,User_id
0,49.00,Male,4700.00,3.00,Basic,NaN,3,NaN,0.00,a271aa56-bcfc-4f0a-91f7-b773de6b86a4
1,69.00,Female,6107.00,2.00,Standard,NaN,1,2.14,0.00,fe14b048-770b-4619-8ec6-485b0969ae31
2,44.00,Male,6374.00,3.00,Premium,1.00,1,1.91,1.00,0283dffd-6684-4a05-9c6f-c02098a6849d
3,26.00,Male,NaN,5.00,Basic,NaN,2,NaN,NaN,c316f2f5-3403-4a0a-82c2-c98e4b3379d2
4,18.00,NaN,1948.00,3.00,Basic,NaN,3,3.88,0.00,14879bcb-1977-4ad8-a7f9-6aa9bf7b872f
...,...,...,...,...,...,...,...,...,...,...
77879,45.00,Male,NaN,NaN,Basic,NaN,5,2.20,0.00,6b51fe0d-d4a1-41ed-a55c-d05f5c961e3c
77880,47.00,Female,6176.00,NaN,Premium,NaN,2,4.54,0.00,59d70167-6be0-474c-b71e-14153205b44f
77881,64.00,Female,NaN,5.00,Premium,NaN,1,4.34,1.00,a7d5ed26-78a4-4e70-bd04-0fec883e56aa
77882,58.00,Male,1314.00,5.00,Standard,NaN,4,NaN,0.00,472e9e70-87ca-4c5e-bfee-fa9f42b54f51


# Etapas de Desenvolvimento

Para te ajudar nesse processo, detalhar o processo nas etapas a seguir:

## Etapa 01) Análise exploratória dos dados (Data Understanding)

a. Carregue a base de dados;

b. Realize uma descrição estatística dos dados;

c. Verifique os tipos de dados

d. Verifique a quantidade de valores faltantes

### Análise Descritiva do Dataset

In [11]:
df_informations(df)

<H3 style='text-align:left;float:lfet;'>Informações sobre o Dataset

<H5> Dataset tem 77884 linhas e 10 colunas. Tem 12403 linhas duplicadas, o que representa 15.92% do total de linhas. Sobre o dataset, temos:

,Not Null,Null,Perce Null,Unique,Dtype
Age,66607,11277,14.48%,52,float64
Gender,63368,14516,18.64%,2,object
Time_on_platform,56464,21420,27.50%,8721,float64
Devices_connected,52696,25188,32.34%,5,float64
Subscription_type,52663,25221,32.38%,3,object
Num_streaming_services,9572,68312,87.71%,4,float64
Num_active_profiles,77884,0,0.00%,5,int64
Avg_rating,60614,17270,22.17%,401,float64
Churned,61148,16736,21.49%,2,float64
User_id,77884,0,0.00%,65481,object


<H3 style='text-align:left;float:lfet;'>Sobre Dtypes, temos:

,Dtype,Count,Perce
0,float64,6,60.00%
1,object,3,30.00%
2,int64,1,10.00%


<H3 style='text-align:left;float:lfet;'>Estatística Descritiva

,count,mean,std,min,1%,25%,50%,75%,99%,max
Age,66607.00,43.51,15.04,18.00,18.00,30.00,43.00,57.00,69.00,69.00
Time_on_platform,56464.00,4385.85,2526.58,0.00,82.00,2196.00,4384.00,6573.00,8675.00,8759.00
Devices_connected,52696.00,3.00,1.41,1.00,1.00,2.00,3.00,4.00,5.00,5.00
Num_streaming_services,9572.00,2.50,1.12,1.00,1.00,1.00,2.00,4.00,4.00,4.00
Num_active_profiles,77884.00,3.00,1.41,1.00,1.00,2.00,3.00,4.00,5.00,5.00
Avg_rating,60614.00,3.00,1.15,1.00,1.04,1.99,3.01,4.00,4.96,5.00
Churned,61148.00,0.24,0.43,0.00,0.00,0.00,0.00,0.00,1.00,1.00


As informações **.describe** podem ser úteis para entender a dispersão e a tendência central dos dados.

- <font color='chartreuse'>Média</font>: Representa a tendência central dos dados;
- <font color='chartreuse'>Desvio Padrão</font>: Indica a dispersão, sugerindo uma variabilidade dos valores;
- <font color='chartreuse'>Quartis</font>: Fornecem informações sobre a distribuição dos dados ao longo de diferentes partes; e
- <font color='chartreuse'>Valor Mínimo e Máximo</font>: Indicam a faixa em que os dados estão concentrados.

### Análise Gráfica

In [12]:
#plot_distribution_and_boxplot(df)

## Etapa 02) Tratamento dos Dados (Data Preparation)
1. Substituir valores “NaN” por 0 Colunas → Time_on_platform, Num_streaming_services, Churned, Avg_rating, Devices_connected
   
2. Dropar linhas nulas nas colunas Gender, Subscription_type e Age
   
3. Transformando valores churned 0 e 1 por No e Yes
   
4. Transformando valores floats em valores inteiros

In [13]:
df

,Age,Gender,Time_on_platform,Devices_connected,Subscription_type,Num_streaming_services,Num_active_profiles,Avg_rating,Churned,User_id
0,49.00,Male,4700.00,3.00,Basic,NaN,3,NaN,0.00,a271aa56-bcfc-4f0a-91f7-b773de6b86a4
1,69.00,Female,6107.00,2.00,Standard,NaN,1,2.14,0.00,fe14b048-770b-4619-8ec6-485b0969ae31
2,44.00,Male,6374.00,3.00,Premium,1.00,1,1.91,1.00,0283dffd-6684-4a05-9c6f-c02098a6849d
3,26.00,Male,NaN,5.00,Basic,NaN,2,NaN,NaN,c316f2f5-3403-4a0a-82c2-c98e4b3379d2
4,18.00,NaN,1948.00,3.00,Basic,NaN,3,3.88,0.00,14879bcb-1977-4ad8-a7f9-6aa9bf7b872f
...,...,...,...,...,...,...,...,...,...,...
77879,45.00,Male,NaN,NaN,Basic,NaN,5,2.20,0.00,6b51fe0d-d4a1-41ed-a55c-d05f5c961e3c
77880,47.00,Female,6176.00,NaN,Premium,NaN,2,4.54,0.00,59d70167-6be0-474c-b71e-14153205b44f
77881,64.00,Female,NaN,5.00,Premium,NaN,1,4.34,1.00,a7d5ed26-78a4-4e70-bd04-0fec883e56aa
77882,58.00,Male,1314.00,5.00,Standard,NaN,4,NaN,0.00,472e9e70-87ca-4c5e-bfee-fa9f42b54f51


In [14]:
# 1. Substituir valores “NaN” por 0 nas colunas especificadas
cols_to_fill = ['Time_on_platform', 'Num_streaming_services', 'Churned', 'Avg_rating', 'Devices_connected']
df[cols_to_fill] = df[cols_to_fill].fillna(0)

# 2. Dropar linhas nulas nas colunas Gender, Subscription_type e Age
cols_to_check = ['Gender', 'Subscription_type', 'Age']
df = df.dropna(subset=cols_to_check)

# 2.1 Dropar as linhas duplicadas
df.drop_duplicates(inplace=True)

# 3. Transformando valores churned 0 e 1 por No e Yes
df['Churned_str'] = df['Churned'].map({0: 'No', 1: 'Yes'})

# 4. Transformando valores floats em valores inteiros
# Primeiro, precisamos identificar as colunas com valores float. 
float_cols = df.select_dtypes(include='float64').columns

# Depois, transformamos cada uma delas em inteiros.
for col in float_cols:
    df[col] = df[col].astype(int)

# Verificamos o resultado
df.head()


,Age,Gender,Time_on_platform,Devices_connected,Subscription_type,Num_streaming_services,Num_active_profiles,Avg_rating,Churned,User_id,Churned_str
0,49,Male,4700,3,Basic,0,3,0,0,a271aa56-bcfc-4f0a-91f7-b773de6b86a4,No
1,69,Female,6107,2,Standard,0,1,2,0,fe14b048-770b-4619-8ec6-485b0969ae31,No
2,44,Male,6374,3,Premium,1,1,1,1,0283dffd-6684-4a05-9c6f-c02098a6849d,Yes
3,26,Male,0,5,Basic,0,2,0,0,c316f2f5-3403-4a0a-82c2-c98e4b3379d2,No
5,54,Male,3024,2,Basic,0,1,0,0,a1df3a13-9255-4d00-8a9d-20565fefaab9,No


In [15]:
df_informations(df)

<H3 style='text-align:left;float:lfet;'>Informações sobre o Dataset

<H5> Dataset tem 30739 linhas e 11 colunas. Não possui linhas duplicadas. Sobre o dataset, temos:

,Not Null,Null,Perce Null,Unique,Dtype
Age,30739,0,0.00%,52,int32
Gender,30739,0,0.00%,2,object
Time_on_platform,30739,0,0.00%,8069,int32
Devices_connected,30739,0,0.00%,6,int32
Subscription_type,30739,0,0.00%,3,object
Num_streaming_services,30739,0,0.00%,5,int32
Num_active_profiles,30739,0,0.00%,5,int64
Avg_rating,30739,0,0.00%,6,int32
Churned,30739,0,0.00%,2,int32
User_id,30739,0,0.00%,30739,object


<H3 style='text-align:left;float:lfet;'>Sobre Dtypes, temos:

,Dtype,Count,Perce
0,int32,6,55.00%
1,object,4,36.00%
2,int64,1,9.00%


<H3 style='text-align:left;float:lfet;'>Estatística Descritiva

,count,mean,std,min,1%,25%,50%,75%,99%,max
Age,30739.00,43.52,15.09,18.00,18.00,30.00,43.00,57.00,69.00,69.00
Time_on_platform,30739.00,3171.18,2908.45,0.00,0.00,0.00,2713.00,5741.00,8634.62,8759.00
Devices_connected,30739.00,2.04,1.82,0.00,0.00,0.00,2.00,4.00,5.00,5.00
Num_streaming_services,30739.00,0.31,0.91,0.00,0.00,0.00,0.00,0.00,4.00,4.00
Num_active_profiles,30739.00,2.99,1.41,1.00,1.00,2.00,3.00,4.00,5.00,5.00
Avg_rating,30739.00,1.95,1.43,0.00,0.00,1.00,2.00,3.00,4.00,5.00
Churned,30739.00,0.19,0.40,0.00,0.00,0.00,0.00,0.00,1.00,1.00


In [16]:
col_categ = list(df.select_dtypes("object").columns)
col_num = list(df.select_dtypes(np.number).columns)

### Correlação

#### Definição de Correlação

A correlação é uma medida estatística que *avalia a relação entre duas variáveis*. Ela indica a direção e a intensidade dessa relação, ou seja, se as variáveis se movem em conjunto (correlação positiva) ou de forma oposta (correlação negativa), e o quão forte essa relação é.

A força da correlação pode ser definida com base no valor do coeficiente de correlação, que varia de -1 a +1. Aqui estão as interpretações comuns para determinar a força da correlação:

1. <font color='yellow'>**Correlação Fraca**:</font>
    * Quando o coeficiente de correlação está próximo de 0, a correlação é considerada fraca.
    * Seja ela positiva ou negativa, a relação entre as variáveis é considerada fraca se o coeficiente de correlação estiver próximo de zero (por exemplo, entre -0,3 e 0,3).
    * Nesse caso, as variáveis têm uma associação limitada e seus movimentos não são consistentes.
<br></br>
2. <font color='orange'>**Correlação Média**:</font>
    * Uma correlação é considerada média quando o coeficiente de correlação está em torno de -0,5 a -0,3 ou de 0,3 a 0,5.
    * A relação entre as variáveis é moderada e mostra algum grau de consistência em seus movimentos.
    * Uma correlação média indica que as variáveis têm alguma influência mútua, mas não é uma relação forte.
<br></br>
3. <font color='chartreuse'>**Correlação Forte**:</font>
    * A correlação é considerada forte quando o coeficiente de correlação está próximo de -1 ou 1.
    * Uma correlação positiva forte (próxima de +1) indica que as variáveis estão fortemente relacionadas e tendem a se mover na mesma direção.
    * Uma correlação negativa forte (próxima de -1) indica que as variáveis estão fortemente relacionadas, mas se movem em direções opostas.
    * Nesses casos, as variáveis têm uma associação consistente e os movimentos de uma variável estão altamente relacionados aos movimentos da outra.


É importante destacar que a força da correlação pode variar de acordo com o contexto e o domínio dos dados. Além disso, a correlação não implica causalidade direta, ou seja, apenas porque duas variáveis estão correlacionadas, não significa que uma causa a outra.

#### Matriz de Correlação

In [17]:
# Matriz de Correlação
correlation_matrix = df[col_num].corr()

# Cria o gráfico de heatmap
fig = go.Figure(data=go.Heatmap(
                   z=correlation_matrix,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='Viridis'))

# Adiciona anotações
annotations = []
for i, row in enumerate(correlation_matrix.values):
    for j, val in enumerate(row):
        annotations.append(go.layout.Annotation(text=str(round(val, 2)), x=correlation_matrix.columns[j], y=correlation_matrix.columns[i], 
                                                showarrow=False, font=dict(color='white')))

# Atualiza layout
fig.update_layout(title='Matriz de Correlação',
                  annotations=annotations,
                  height=500, width=600)

fig.show()


Explicado o que é correlação, podemos observar que:

1. A correlação entre "youtube" e "sales" é de 0.78, o que indica uma correlação positiva forte. Isso sugere que há uma forte relação entre o investimento no YouTube e as vendas. Aumento nas vendas tendem a estar associados ao aumento de investimento no YouTube.

2. A correlação entre "facebook" e "sales" é de 0.60, o que indica uma correlação positiva forte. Isso sugere que há uma relação forte entre o investimento no Facebook e as vendas. Aumentos nas vendas tendem a estar associados ao aumento de investimento no Facebook.

3. A correlação entre "newspaper" e "sales" é de 0.25, o que indica uma correlação positiva fraca. Isso sugere que há uma relação fraca entre o investimento em newspaper e as vendas. A influência dos newspaper nas vendas é limitada, pois a correlação é relativamente baixa.

#### Scatter plot: Sales vs Investimento

In [ ]:
# Reestrutura os dados para um formato longo
df_melted = df.melt(id_vars='sales', var_name='platform', value_name='investment')

# Cria um dicionário com os símbolos para cada plataforma
symbols = {'youtube': 'circle', 'facebook': 'diamond', 'newspaper': 'square'}

# Cria o gráfico de dispersão
fig = px.scatter(df_melted, x='investment', y='sales', color='platform', 
                 symbol=df_melted['platform'].map(symbols), 
                 title='Scatter plot: Sales vs Investimento',
                 labels={'investment':'Investimento', 'sales':'Sales'}, 
                 hover_data=['platform', 'investment', 'sales'])

fig.show()


Analisando o Scatter plot é possível observar que é necessário desprender ou investir muito mais dinheiro no Youtube para ter o mesmo retorno que as demais plataformas. Em contra partida o Facebook e Newspaper possuem uma distribuição semelhante, onde a relação Sales vs Investimento é mais vantajosa, sendo necessário despender menos dinheiro para um retorno maior.

### Outliers

#### Definição de Outliers

Outliers são valores em um conjunto de dados que são significativamente diferentes dos outros. Eles são valores extremos que se desviam da média e das medidas padrão do conjunto de dados.

Para identificar outliers em um conjunto de dados, você pode usar vários métodos. Aqui estão algumas opções populares:

1. **Gráficos Boxplot**
   
2. **Z-Score**
   
3. **Desvio absoluto mediano (MAD)**
   
4. <font color='orange'>**Regra do IQR (Interquartile Range)**</font>

A análise de outliers é crucial para uma análise de dados precisa, pois outliers podem distorcer os resultados. Boxplots são uma ferramenta gráfica comum para identificar outliers, baseando-se no Intervalo Interquartil (IQR).

Apesar da facilidade de identificar outliers visualmente em boxplots, elaborei uma função para calcular numericamente o IQR e os limites de outliers. Essa função nos permite entender melhor o processo de detecção de outliers.

Apesar de útil, não é necessário calcular manualmente esses valores sempre. Ferramentas como boxplots fazem isso automaticamente, facilitando a identificação de outliers. Na sequência, utilizaremos boxplots para visualizar a presença de outliers em nossos dados.

O IQR é a diferença entre o terceiro quartil (Q3) e o primeiro quartil (Q1) dos dados. Qualquer ponto de dados que esteja abaixo de <font color='orange'>**Q1-1.5IQR**</font> ou acima de <font color='orange'>**Q3+1.5IQR**</font> pode ser considerado um outlier.

#### Cálculo dos Outliers

In [ ]:
# Usando a função no DataFrame df
iqr_df = calculate_iqr(df[['youtube', 'facebook', 'newspaper', 'sales']])
iqr_df


Os outliers são valores que desviam notavelmente do restante dos dados. No contexto do IQR, outliers são aqueles valores que estão **abaixo** do "Lower Bound" ou **acima** do "Upper Bound". Assim, na tabela IQR que geramos, qualquer valor da coluna "Min" menor que o respectivo "Lower Bound" e qualquer valor da coluna "Max" maior que o respectivo "Upper Bound" são considerados outliers.

In [ ]:
# Usando a função com nosso DataFrame df
plot_boxplots(df[['youtube', 'facebook', 'newspaper', 'sales']])


Ao analisar os boxplots das nossas variáveis 'youtube', 'facebook', 'newspaper' e 'sales', notamos que apenas a variável 'newspaper' apresenta outliers. No entanto, essa quantidade de outliers parece ser pequena e, portanto, é improvável que cause um impacto significativo em nossa análise posterior ou distorça nossos resultados.

## Etapa 03)  Modelagem dos Dados - Regressão Logística
a. Definir variáveis X e y para o modelo

b. Realizar o .fit do modelo

c. Separar em train e test

d. Realizar a modelagem

e. Plotar matrix confusão

f. Printar métricas

### Definição de Regressão Linear

#### Regressão Linear Simples

A regressão linear é uma técnica estatística que tenta modelar a relação entre uma variável dependente (também conhecida como variável de resposta) e uma ou mais variáveis independentes (também conhecidas como variáveis preditoras) por meio de uma equação linear. Na regressão linear simples (com uma variável independente), essa relação é modelada como uma linha reta (daí o termo "linear"). Em regressão linear múltipla (com mais de uma variável independente), essa relação é modelada como um hiperplano.

Vamos pensar na regressão linear simples por um momento. A equação básica que estamos tentando resolver é:

<font color='orange'>**<center>y = mx + b</center>**</font>

Onde:
- <font color='orange'>***y***</font> é a variável dependente; 
- <font color='orange'>***x***</font> é a variável independente; 
- <font color='orange'>***m***</font> é a inclinação da linha de regressão (representa o efeito de <font color='orange'>***x***</font> sobre <font color='orange'>***y***</font>);
- <font color='orange'>***b***</font> é a interceptação (representa o valor de <font color='orange'>***y***</font> quando <font color='orange'>***x***</font> é 0).

A regressão linear, em sua essência, é um método para encontrar os melhores valores para m e b. E quando dizemos "melhor", estamos falando em termos de minimizar a distância entre a linha de regressão (os valores previstos de y para qualquer valor de x) e os pontos de dados reais.

Abaixo eu criei um GIF que demonstra o comportamento da reta quando alteramos os valores de <font color='orange'>***m***</font> e <font color='orange'>***b***</font> de um Regressão Linear Simples (com <font color='RED'>**UMA**</font> variável independente).

In [ ]:
Image(url='GIF Linear Regression.gif')

#### Regressão Linear Múltipla

Passando para a regressão linear múltipla, a ideia básica é a mesma, mas a matemática se torna um pouco mais complicada. Agora, em vez de termos apenas um m e um b, temos um coeficiente para cada variável independente e uma interceptação. Então, nossa equação se parece com:

<font color='orange'>**<center>y = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ + ε</center>**</font>

Onde:
- <font color='orange'>***y***</font> é a variável dependente, ou seja, a variável que estamos tentando prever ou estimar; 
- <font color='orange'>***β₀***</font> é o termo de interceptação. Ele representa o valor esperado de Y quando todas as variáveis independentes (Xs) são iguais a zero; 
- <font color='orange'>***β₁, β₂, ..., βₙ***</font> são os coeficientes de regressão. Eles representam a mudança esperada na variável dependente (Y) para cada mudança de uma unidade na respectiva variável independente, mantendo todas as outras variáveis independentes constantes;
- <font color='orange'>***x₁, x₂, ..., xₙ***</font> são as variáveis independentes, ou seja, as variáveis que usamos para prever ou estimar Y.
- <font color='orange'>***ε***</font> é o termo de erro, também conhecido como resíduos. Ele representa a diferença entre o valor real e o valor previsto de Y.

Quando estamos trabalhando com regressão linear múltipla, estamos tentando ajustar um modelo a um conjunto de dados que tem múltiplas variáveis independentes, que resultam em uma dimensão adicional para cada variável adicional.

Uma maneira de visualizar isso é imaginar que estamos tentando ajustar um plano em um espaço tridimensional em vez de uma linha em um espaço bidimensional. Se você tiver duas variáveis independentes, poderá visualizá-las ao longo de dois eixos (digamos x e y), e a variável dependente ao longo de um terceiro eixo (digamos z). Nesse caso, o "ajuste" mais próximo dos pontos de dados seria um plano que minimiza a distância entre os pontos de dados e o próprio plano.

**_Agora, se houvesse mais de duas variáveis independentes, o mesmo conceito se aplica, mas seria mais difícil visualizar porque estaríamos trabalhando em mais de três dimensões. No entanto, a matemática subjacente é a mesma: estamos tentando encontrar o "plano" (ou, mais tecnicamente, o hiperplano) que minimiza a distância entre os pontos de dados e o próprio hiperplano._**

Abaixo eu criei um gráfico 3D para demonstrar o comportamento de uma Regressão Linear Múltipla (com <font color='RED'>**DUAS**</font> variável independente).

In [ ]:
# Gerando dados aleatórios para o exemplo
np.random.seed(0)
x1 = np.random.rand(100, 1)
x2 = np.random.rand(100, 1)
y2 = 3*x1 + 2*x2 + np.random.rand(100, 1)

# Modelo de regressão linear múltipla
model = LinearRegression()
model.fit(np.hstack([x1, x2]), y2)

# Você precisa chamar a função com os coeficientes obtidos do modelo.
update_graph2(model.coef_[0][0], model.coef_[0][1], model.intercept_[0])

### Separando em conjuntos de Treino e Teste

In [ ]:
# Crie um novo dataframe df_model
df_model = df[['youtube', 'facebook', 'newspaper', 'sales']].copy()

# Antes da divisão, criei uma nova coluna chamada 'split'
df_model['split'] = 'train'

# Dividindo o conjunto de dados
X = df_model[['youtube', 'facebook', 'newspaper']]
y = df_model['sales']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Indique que os dados em X_test estão na partição de teste
df_model.loc[X_test.index, 'split'] = 'test'

# Verificando os tamanhos dos conjuntos de treino e teste
print("Treino:", X_train.shape, y_train.shape)
print("Teste:", X_test.shape, y_test.shape)

In [ ]:
df_model

## Etapa 04) Modelagem dos Dados - Tunning
a. Definir variáveis X e y para o modelo

b. Realizar o .fit do modelo

c. Separar em train e test

d. Realizar a modelagem

e. Plotar matrix confusão

f. Printar métricas

#### Treinando o Modelo

O método ***.fit()*** é usado para treinar ou ajustar o modelo aos seus dados de treinamento. Em outras palavras, é aqui que o modelo aprende os padrões nos dados.

No caso de um modelo de regressão linear, o processo de ajuste envolve aprender os coeficientes que minimizam a soma dos erros quadrados entre os valores reais e os valores previstos pelo modelo. Para isso, o modelo usa o método dos mínimos quadrados.

Então, quando chamamos ***model.fit(X_train, y_train)***, estamos dizendo ao modelo para encontrar os melhores parâmetros (coeficientes de regressão) que mapeiam as variáveis independentes ***X_train*** para a variável dependente ***y_train*** com o menor erro.

Uma vez que o modelo foi treinado usando ***.fit()***, ele pode ser usado para fazer previsões em novos dados usando o método ***.predict()***. Nesse ponto, o modelo aplica os coeficientes de regressão que aprendeu durante o treinamento para prever a variável dependente a partir das variáveis independentes nos novos dados.

In [ ]:
# Treinando o modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Crie as previsões
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

Ao aplicar o método ***predict*** aos dados de treinamento **(X_train)**, estamos gerando previsões para os dados que foram usados para treinar o modelo. Essas previsões podem ser comparadas com os valores reais **(y_train)** para avaliar o quão bem o nosso modelo aprendeu os padrões nos dados de treinamento. Esta é uma forma de avaliar a precisão do treinamento do nosso modelo.

Por outro lado, quando aplicamos o método ***predict*** aos dados de teste **(X_test)**, estamos gerando previsões para novos dados que o modelo ainda não viu. Comparando essas previsões com os valores reais **(y_test)**, podemos avaliar a capacidade do nosso modelo de generalizar para novos dados. Esta é uma medida da precisão de teste do nosso modelo.

In [ ]:
# Adicionando as previsões ao dataframe
df_model.loc[X_train.index, 'prediction'] = y_train_pred
df_model.loc[X_test.index, 'prediction'] = y_test_pred

df_model

In [ ]:
tabela = pd.DataFrame()
tabela["y_teste"] = y_test.values
tabela["Previsoes Regressao Linear"] = y_test_pred

# Calcula os resíduos
tabela["Residuos"] = tabela["y_teste"] - tabela["Previsoes Regressao Linear"]

# Cria o objeto Figure
fig = go.Figure()

# Adiciona uma linha para os valores reais
fig.add_trace(go.Scatter(y=tabela['y_teste'], mode='lines', name='Valores Reais'))

# Adiciona uma linha para os valores previstos
fig.add_trace(go.Scatter(y=tabela['Previsoes Regressao Linear'], mode='lines', name='Previsoes Regressao Linear'))

# Adiciona uma linha para os resíduos
fig.add_trace(go.Scatter(y=tabela['Residuos'], mode='lines', name='Resíduos'))

# Adiciona título e rótulos de eixo
fig.update_layout(
    title='Valores Reais vs Previstos vs Resíduos',
    xaxis=dict(title='Índice'),
    yaxis=dict(title='Vendas')
)

# Mostra o gráfico
fig.show()


**Interpretação do Gráfico**

* A linha "Valores Reais" mostra os valores reais de venda que foram observados no conjunto de dados de teste.

* A linha "Previsões da Regressão Linear" mostra os valores de venda que foram previstos pelo nosso modelo de regressão linear a partir das variáveis independentes no conjunto de dados de teste.

* A linha "Resíduos" mostra a diferença entre os valores reais e os previstos, também conhecida como erro de previsão ou resíduo.

Comparando as linhas de "Valores Reais" e "Previsões da Regressão Linear", podemos ter uma ideia de quão bem o nosso modelo de regressão está funcionando. Idealmente, queremos que essas duas linhas estejam o mais próximas possível, o que indicaria que o nosso modelo está fazendo boas previsões.

A linha de "Resíduos" nos permite ver a magnitude dos erros de previsão. Valores próximos de zero indicam boas previsões, enquanto valores grandes (positivos ou negativos) indicam previsões ruins. Se vemos muitos valores grandes na linha de "Resíduos", isso pode ser um sinal de que o nosso modelo de regressão pode ser melhorado.

In [ ]:
# Scatter plot
fig = px.scatter(
    df_model, x='sales', y='prediction',
    marginal_x='histogram', marginal_y='histogram',
    color='split', trendline='ols'
)
fig.update_traces(histnorm='probability', selector={'type':'histogram'})
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=y.min(), y0=y.min(),
    x1=y.max(), y1=y.max()
)

fig.show()

**Interpretação do Gráfico**

* Os pontos representam as observações. Os pontos coloridos representam se a observação pertence ao conjunto de treinamento (azul) ou ao conjunto de teste (vermelho).
  
* O histograma na margem superior mostra a distribuição das vendas reais e o histograma na margem direita mostra a distribuição das vendas previstas.
  
* A linha tracejada preta representa a linha de perfeita igualdade (y = x), ou seja, os pontos que caem nessa linha são aqueles para os quais a venda real é igual à venda prevista. Idealmente, queremos que nossos pontos fiquem o mais próximos possível dessa linha.
  
* A linha de tendência azul (ols - ordinary least squares) é a linha de melhor ajuste para o conjunto de treinamento, minimizando a soma dos quadrados dos resíduos (ou erros).
  
* A linha de tendência vermelha é a linha de melhor ajuste para o conjunto de teste.


No geral, este gráfico nos permite ver quão bem o nosso modelo de regressão está prevendo as vendas. Se o modelo é bom, os pontos devem estar próximos da linha tracejada preta (linha de perfeita igualdade) e as linhas de tendência azul e vermelha devem ser semelhantes.

#### Avaliação do Modelo

O Root Mean Square Error (RMSE) e o coeficiente de determinação (R²) são duas métricas comumente usadas para avaliar a performance de um modelo de regressão.

##### Root Mean Square Error (RMSE)

O RMSE é uma medida de erro que compara os valores previstos por um modelo com os valores reais. Ele é calculado ao se tomar a média dos quadrados dos erros, e em seguida tirando a raiz quadrada. Essa métrica dá uma ideia da quantidade de erro que o sistema normalmente comete em suas previsões, com um maior peso para erros maiores. Uma vantagem do RMSE é que o erro é expresso na mesma unidade que a variável de saída (y).

Passo a passo de como o RMSE é calculado:

1. <font color='orange'>**Erro de Previsão:**</font> Para cada ponto de dados, você calcula a diferença entre a previsão do modelo e o valor real. Se o seu modelo previu um valor de 10 para uma determinada observação, mas o valor real é 12, o erro de previsão é -2 (10-12). Se a previsão fosse 14, o erro seria 2 (14-12).

2. <font color='orange'>**Quadrado do Erro de Previsão:**</font> Em seguida, você eleva ao quadrado cada erro de previsão. Fazemos isso por dois motivos: 
   
   * <font color='orange'>Primeiro:</font> para garantir que todos os erros sejam positivos (a diferença -2 e a diferença 2 têm o mesmo impacto). 
   * <font color='orange'>Segundo:</font> para dar mais peso a erros maiores. No exemplo anterior, o quadrado de -2 é 4 e o quadrado de 2 é 4.
<br></br>
3. <font color='orange'>**Média dos Quadrados dos Erros:**</font> Em seguida, você calcula a média de todos os quadrados dos erros de previsão. Isso é conhecido como ***Mean Squared Error (MSE)***. Se você teve 5 previsões, e os quadrados dos erros foram [4, 4, 9, 4, 1], o MSE seria a soma desses números dividida por 5, que é 22/5 = 4.4.

4. <font color='orange'>**Raiz Quadrada da Média dos Quadrados dos Erros:**</font> Finalmente, você tira a raiz quadrada do MSE para obter o RMSE e para trazer o erro de volta às unidades originais do output. Ao elevar ao quadrado os erros, como fazemos ao calcular o MSE, estamos efetivamente colocando os erros em termos de suas unidades ao quadrado. Isso pode ser um pouco abstrato e difícil de interpretar. No exemplo anterior, a raiz quadrada de 4.4 é aproximadamente 2.097.

Portanto, um RMSE de 2.097 significaria que, em média, as previsões do modelo estão cerca de 2.097 unidades distantes dos valores reais. Quanto menor o RMSE, melhor o modelo é capaz de prever os dados.

##### R²

O coeficiente de determinação, ou R², é uma medida estatística que indica a proporção da variação na variável dependente que é previsível a partir da(s) variável(is) independente(s). Em outras palavras, R² é uma medida de quão bem as previsões do modelo se ajustam aos dados reais. O valor de R² varia entre 0 e 1, onde 1 indica que o modelo explica toda a variabilidade dos dados em torno da média.

Passo a passo de como o R² é calculado:

<font color='orange'>**<center>R² = 1 - (Soma dos Quadrados dos Resíduos / Soma Total dos Quadrados)</center>**</font>

Vamos supor que temos os seguintes valores reais de Y: [3, -0.5, 2, 7] e suas previsões correspondentes do modelo são: [2.5, 0.0, 2, 8].

1. <font color='orange'>**Calcule a média de Y:**</font> neste caso, a média é (3 - 0.5 + 2 + 7)/4 = 2.875.

2. <font color='orange'>**Calcule a soma total dos quadrados (SST):**</font> que é a soma das diferenças quadradas entre cada valor de Y e a média de Y. Para nosso exemplo, a SST é (3-2.875)² + (-0.5-2.875)² + (2-2.875)² + (7-2.875)² = 30.375.

3. <font color='orange'>**Calcule a soma dos quadrados dos resíduos (SSR):**</font> que é a soma das diferenças quadradas entre cada valor de Y e a previsão correspondente do modelo. Para nosso exemplo, a SSR é (3-2.5)² + (-0.5-0.0)² + (2-2)² + (7-8)² = 1.5.

4. <font color='orange'>**R²:**</font> é calculado como 1 - (SSR/SST). Então, nosso R² é 1 - (1.5/30.375) = 0.951.

Um R² de 0.951 indica que 95.1% da variação total em Y é explicada pelo modelo, o que é um bom ajuste.

### Conclusão Etapa 4 - Avaliação do Modelo

In [ ]:
# Calcular o RMSE
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"RMSE no conjunto de treinamento: {rmse_train:.2f}")
print(f"RMSE no conjunto de teste: {rmse_test:.2f}")

In [ ]:
# Calcular o R²
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"R² no conjunto de treinamento: {r2_train:.2f}")
print(f"R² no conjunto de teste: {r2_test:.2f}")

In [ ]:
# Obtendo os coeficientes
coef = model.coef_

# Transformando em porcentagem
coef_percent = coef * 100

# Formatando os coeficientes para remover a notação científica
formatted_coef = ['{:.2f}%'.format(i) for i in coef_percent]

# Associando cada coeficiente à sua feature correspondente
features = ['youtube', 'facebook', 'newspaper']
results = dict(zip(features, formatted_coef))

# Imprimindo os resultados
for feature, coef in results.items():
    print(f'{feature} - {coef}')


Com base nos resultados, podemos concluir que:

1. O modelo apresentou bom desempenho, com RMSE de 1.89 no treino e 2.36 no teste, indicando pequena diferença entre os valores previstos e reais.

2. O alto R² (0.91 treino e 0.87 teste) sugere que o modelo explica uma grande parte da variação nos dados.

3. Em relação aos coeficientes:

   * Para cada aumento de uma unidade em youtube, espera-se que as sales aumentem, em média, 4.42% (assumindo que todas as outras variáveis permaneçam constantes).
  
   * Para cada aumento de uma unidade em facebook, espera-se que as sales aumentem, em média, 19.45% (assumindo que todas as outras variáveis permaneçam constantes).
  
   * Para cada aumento de uma unidade em newspaper, espera-se que as sales diminuam, em média, 0.49% (assumindo que todas as outras variáveis permaneçam constantes).

Em suma, o modelo de regressão linear é eficaz em prever as vendas baseado nos investimentos em publicidade e os resultados da análise sugerem que a plataforma do **Facebook** pode ser a mais eficaz para gerar vendas a partir do investimento em publicidade, dada a sua maior correlação com as vendas (representada pelo coeficiente de 19.45%). Isso sugere que cada unidade de aumento no investimento em publicidade no Facebook poderia resultar, em média, em um aumento de 19.45% nas vendas, supondo que todas as outras variáveis permaneçam constantes.

## Etapa 05) Modelagem dos Dados - Random Forest
a. Realizar a montagem do grid search

b. Realizar o .fit do modelo

c. Realizar o Tunning

d. Realizar a modelagem

e. Plotar matrix confusão

f. Printar métricas